# 456 `132` Pattern

我真傻。真的。

实际上我用三个函数来回嵌套 试图用 lru_cache 的办法很蠢。基本上没可能 Hit。在不 Hit 的条件下这么写还不如套三个 For 循环。

这里的复杂度实际上是可以大大降低。

首先，构造一个最小堆，和一个排序好的栈。

min_arr[i] 里放着的是 nums[:i] 里的最小值。

然后，我们反向遍历 nums。在我们遍历到 nums[i] 的时候，一旦发现她大于 min_arr[i]，也就是说 nums[:i] 里至少存在一个比 nums[i] 小的值，我们就从 stack 中通过弹出的方式找出最小的大于 min_arr[i] 的值。假如这个值同时也小于 nums[j]，那么我们就找到了那个 132 里的 2。假如这个值不是「2」的话，我们还是要把 nums[i] 给加入栈中。

留意到我们在把 nums[i] 入栈之前，已经把所有小于她的值（小于 min_arr[j] 的）都给弹出去了。
所以一定有 nums[i]<=stack[-1]，因此每次加入的值总是不更小。

留意到我们的 min_arr 是自左向右算的，要找到的是 132 里的「1」；而较大的「2」和「3」 则是自右向左逼近的。

In [ ]:
class Solution(object):
    def find132pattern(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        # use a min heap instead of stack to save the potential k value
        # https://leetcode.com/problems/132-pattern/discuss/432266/Python-Different-approach-explained-(array-%2B-heap).-O(n-log-n)
        if len(nums)<3:
            return False
        min_arr = [None for _  in range(len(nums))]
        min_arr[0] = nums[0]
        stack = [] # a sorted stack by default
        for i in range(1,len(nums)):
            min_arr[i] = min(min_arr[i-1],nums[i])
        for j in range(len(nums)-1,-1,-1):
            if nums[j]>min_arr[j]:
                while stack and stack[-1]<=min_arr[j]:
                    stack.pop()
                if stack and stack[-1]<nums[j]:
                    return True
                #nums[j]<=stack[-1], so the stack remains sorted by default
                stack.append(nums[j]) 
            else:
                #nums[j]==min_arr[j]
                #so that nums[j]<=min_arr[k], for k<j, because min_arr is in
                #descending order
                pass     
        return False

written by @petterbj

So after I read a lot of posts about the stack solution and still couldn't understand any of them.
I decided to wirte a post on my own and break down every single details of it.

Before you start reading it, I have to tell you this post is about the stack solution. There exists many other solutions in the solution tab of the problem. However, this post only focus on the stack solution.
And I am going to use c++ terms like vector or stack<int> incase you don't know what I am talking about.

Ok, let's get started.

So after first observation of the problem one could come up with a O(n^3) solution right?
If there exists a solution ai, aj, ak for which aj > ak > ai, then aj must exists in the nums vector, right?1 3 2 ajmeans the biggest one.

So we can go through this vector nums, for every nums[i]we check if it is a aj. We look to the left of nums[i] looking for the smallest one as ai, and we look into the right of ajlooking for a number which is smaller than aj and bigger than ai, if we succeed in finding such numbers, then nums[i] is indeed a aj, otherwise, after we go through the entire vector and still could not find any, then there doesn't exists such pattern.
This is a O(n^2) solution.

How can we do better than that?

one little optimization is to go through the vector from left to right first and store the smallest item so far as a min_vector. For everymin_vector[i] is the smallest number found from nums[0]to nums[i]

So we don't need to look into the left any more, simply check the min_vector[i]to get the smallest number. This will take O(n) space, yet still haven't changed the time complexicity.

**Just hear me out. The next bit is very important.
**

When we look to the right and looking for the ak, if we search from the end to the start
like for(j= n-1;j>=0;j--)every time we check if nums[j]is bigger than min_vector[i] if so, we print the nums[j]to the screen, we are going to see a very interesting pattern. The numbers printed to the screen are in descending order, if there exists a number that destroy the descending pattern, then these two numbers along with the min_vector[i] form a solution to the problem, regardless the state of nums[i].

Please allow me to elaborate on that. For the nums[i] to become ajit must be bigger than ai. For nums[j]to become akthen it must also be bigger than ai other wise it is not a 132 pattern. So when we look to the right of nums[i]we only care about the ones bigger than min_vector[i]. For all the numbers to the right of nums[i]that are bigger than min_vector[i]we only need to check if these numbers are in ascending order. If they are not in ascending order, then there exists a solution. the function return true. till now, the time complexicity is still not changed.

If we now take a closer look at nums[i]and nums[i+1]for nums[i]we go through all the element on the right to see if there exists a solution by check if the sequence is in ascending order. and for nums[i+1]we went throught all the numbers on the right to see if they are in ascending order. For these two numbers, we don't need to go through the numbers twice. If fornums[i+1] the numbers are in ascending order. we only need to check if from nums[i]to nums[i+1] is in ascending order and all the numbers on the right are automatically in ascending order. Becauese otherwise we have found a solution already and the function has returned. This operation takes O(1) time.

HaHa, we now we have reduced the time complexicity from O(n^2) to O(n), right?

Not exactly, there is small thing one is likely to overlook that is min_vector[i] min_vector[i+1].
The min_vector is in descending order from left to right. So, for all the number to the right of nums[i]that are greater than min_vector[i] are a little different from the numbers to the right of nums[i+1] that are greater than min_vector[i+1]. becase min_vector[i] is >= min_vector[i+1]

This poses problems, right? Here is where the stack comes in. When we were processing from end all the way to nums[i+1]we can push all the numbers to a stack. For this particular stack. The top one is the smallest, all the way to bottom, the numbers are getting bigger. When the nums[j+1]failed and we move to the nums[j]the stack is basically the same. we only need to pop out all the numbers that are smaller than min_vector[i]and check if nums[i] is smaller than top of the stack. If so, that means nums[i] is not what we are looking for, we push nums[i] to the stack assuming it is bigger than min_vector[i] then we continue to i-1 . If not, we have found a solution and the function return true.

Hence, this is how we can reduce the time complexicity from o(n^2) to O(n) with the use of stack.

P.S. This is longer than I originally thought. Thank you so much for reading all of it.
If you found any thing not very easy to understand please comment below, I will come back from time to time to update it. Thanks.